In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from find_peak_slope import *
from simulate import *
from zacLib import awgn
%matplotlib Qt5

In [2]:
def demo(x,sp,width,slope_th):
    (sm,dsp,new_peak,new_pits,avg_frq,max_frq,
     dev_band,half_left,half_right,half_band) = find_peak_slope(sp,width,slope_th)
     
    plot_all(sp,sm,dsp,slope_th,new_peak,new_pits,avg_frq,
             max_frq,dev_band,half_left,half_right,1,origin=x)

In [10]:
# real data

find_peak_slope_from_rd('sig0.rd',150,20,1,0)
# find_peak_slope_from_rd('sig1.rd',300,30,1,0)
# find_peak_slope_from_rd('sig2.rd',40,10,1,0)
# find_peak_slope_from_rd('sig3.rd',300,30,1,0)
# find_peak_slope_from_rd('siga.rd',300,30,1,0)

,avg_frq,max_frq,slope band,half_band
0,7874,8041,1072,1024
1,9200,9471,1026,1002
2,11131,11169,395,391
3,11700,11813,388,403
4,12122,12129,155,143
5,12544,12545,162,152
6,13242,13383,563,535
7,14225,14187,542,532
8,14937,14937,156,134
9,15347,15352,150,122


In [4]:
# simulate data

# bell shape
# x,sp = bells(noise=0.3)
# demo(x,sp,200,20)

# square
# x,sp = squares(0.5)
# demo(x,sp,100,20)

#stair
# x,sp = stairs(0.5)
# demo(x,sp,200,30)

# random combination
# x,sp = random(noise=0.7)
# demo(x,sp,100,20)